# Web Scrape Cattle Market Reports for a Given Location

![Alt Text](cattle.jpeg)

# Libraries

In [ ]:
import time
import csv
import os
import re
import requests 
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver

# Custom Scraping Methods

In [ ]:
def sanitize_filename(filename):
    return re.sub(r'[<>:"/\\|?*]', '_', filename)

def create_output_directory(location):
    today_date = datetime.today().strftime('%Y-%m-%d')
    directory_name = f"{today_date}_{location}_cattle_sales"
    os.makedirs(directory_name, exist_ok=True)
    return directory_name

def click_link(driver, href):
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, f"//a[contains(@href, '{href.split('auctionId=')[1]}')]"))
    )
    element.click()

def extract_sale_date(soup):
    return soup.find('h3', class_='sc-blLsxD kXGvid').text.strip()

def extract_table_data(soup):
    table = soup.find('table', {'class': 'sc-rZqKh kUEjMi'})
    headers = [header.text.strip() for header in table.find('thead').find('tr').find_all('td')]
    data_rows = table.find('tbody').find_all('tr')
    all_data = [[cell.text.strip() for cell in row.find_all('td')] for row in data_rows]
    return headers, all_data

def write_to_csv(output_dir, date_of_sale, headers, data):
    headers.append('DateOfSale')
    for row in data:
        row.append(date_of_sale)
    sanitized_date = sanitize_filename(date_of_sale)
    csv_path = os.path.join(output_dir, f"{sanitized_date}.csv")
    with open(csv_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(headers)
        writer.writerows(data)

def process_sales_data(driver, href_dict, location):
    output_dir = create_output_directory(location)
    for text, href in href_dict.items():
        if location.lower() in text.lower():
            click_link(driver, href)
            time.sleep(6)
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            date_of_sale = extract_sale_date(soup)
            headers, data = extract_table_data(soup)
            write_to_csv(output_dir, date_of_sale, headers, data)
            time.sleep(3)
            driver.back()
            time.sleep(3)

def fetch_href_dict(driver, url, xpath):
    driver.get(url)
    print(driver.title)
    td_elements = driver.find_elements(By.XPATH, xpath)
    href_dict = {td.text.strip(): td.get_attribute("href") for td in td_elements}
    return href_dict



# Run Web Scraper

In [ ]:
location = "virden"
url = "https://dlms.ca/Report/MarketReport"
xpath = "//td/a"

# Set up WebDriver
driver = webdriver.Chrome()

try:
    # Fetch href_dict
    href_dict = fetch_href_dict(driver, url, xpath)

    # Process sales data
    process_sales_data(driver, href_dict, location)

finally:
    # Quit the WebDriver
    driver.quit()

# Merge CSVs

In [ ]:
import os
import pandas as pd

folder_path = f''


csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

df_list = []

for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)  # Read the CSV file into a dataframe
    df_list.append(df)  # Append the dataframe to the list

combined_df = pd.concat(df_list, ignore_index=True)

output_file_path = f'c:\\combined_{location}_cattle_sales.csv'


combined_df.to_csv(output_file_path, index=False)


In [12]:
combined_df.dtypes

Seq#             int64
Lot#             int64
Pen #           object
Head             int64
Sex             object
Breed           object
Avg. Weight      int64
Total Weight     int64
Price           object
DateOfSale      object
dtype: object